In [ ]:
import torch

In [ ]:
torch.manual_seed(1337)
B, T, C = 4, 8, 2
x = torch.randn(B, T, C)
x.shape

In [ ]:
xbow = torch.zeros((B, T, C))
xbow.shape
for b in range(B):
    for t in range(T):
        xprev = x[b, : t + 1]
        print(b, t, xprev.shape)

        xbow[b, t] = torch.mean(xprev, 0)

xbow.shape

In [ ]:
x[0][:5]

In [ ]:
torch.mean(x[0][:5], axis=0)

In [ ]:
x[0]